In [1]:
import numpy as np
import random
# Definicion de ambiente
class InventoryEnvironment:
    def __init__(self):
        self.products = ['product_A', 'product_B']
        self.max_stock = 10 # Pueden cambiar este número si gustan
        self.demand = {'product_A': [0, 1, 2], 'product_B': [0, 1, 2]}
        self.restock_cost = {'product_A': 5, 'product_B': 7}
        self.sell_price = {'product_A': 10, 'product_B': 15}
        self.state = None 
    def reset(self):
        self.state = {product: random.randint(0, self.max_stock) for product in self.products}
        return self.state
    
    def step(self, action):
        reward = 0
        for product in self.products:
            stock = self.state[product]
            restock = action[product]
            self.state[product] = min(self.max_stock, stock + restock)
            demand = random.choice(self.demand[product])
            sales = min(demand, self.state[product])
            self.state[product] -= sales
            reward += sales * self.sell_price[product] - restock * self.restock_cost[product]
        return self.state, reward
# Init el ambiente
env = InventoryEnvironment()

In [55]:
# Función para generar un episodio
def generate_episode(env, policy, max_steps=10):
    state = env.reset()
    episode = []
    for _ in range(max_steps):
        action = policy(state)
        next_state, reward = env.step(action)
        episode.append((state, action, reward, next_state.copy()))
        state = next_state.copy()
    return episode

# Política de inventario específica (ejemplo simple que reabastece al azar)
def random_policy(state):
    return {product: random.randint(0, 2) for product in state.keys()}



In [56]:
# Generar episodios y recolectar datos
env = InventoryEnvironment()
episodes = [generate_episode(env, random_policy) for _ in range(100)]

# Mostrar un ejemplo de episodio
for step in episodes[0]:
    print(step)

({'product_A': 3, 'product_B': 3}, {'product_A': 2, 'product_B': 2}, -9, {'product_A': 9, 'product_B': 6})
({'product_A': 9, 'product_B': 6}, {'product_A': 2, 'product_B': 1}, 3, {'product_A': 8, 'product_B': 7})
({'product_A': 8, 'product_B': 7}, {'product_A': 0, 'product_B': 0}, 50, {'product_A': 6, 'product_B': 5})
({'product_A': 6, 'product_B': 5}, {'product_A': 2, 'product_B': 2}, 11, {'product_A': 6, 'product_B': 6})
({'product_A': 6, 'product_B': 6}, {'product_A': 0, 'product_B': 0}, 25, {'product_A': 5, 'product_B': 5})
({'product_A': 5, 'product_B': 5}, {'product_A': 1, 'product_B': 1}, 28, {'product_A': 5, 'product_B': 4})
({'product_A': 5, 'product_B': 4}, {'product_A': 0, 'product_B': 2}, 16, {'product_A': 5, 'product_B': 4})
({'product_A': 5, 'product_B': 4}, {'product_A': 1, 'product_B': 0}, 20, {'product_A': 5, 'product_B': 3})
({'product_A': 5, 'product_B': 3}, {'product_A': 1, 'product_B': 0}, 15, {'product_A': 4, 'product_B': 3})
({'product_A': 4, 'product_B': 3}, {'p

In [53]:
def exploring_starts(env, num_episodes=100, max_steps=10):
    episodes = []
    for _ in range(num_episodes):
        state = {product: random.randint(0, env.max_stock) for product in env.products}
        action = {product: random.randint(0, 2) for product in env.products}
        episode = []
        for _ in range(max_steps):
            next_state, reward = env.step(action)
            episode.append((state, action, reward, next_state.copy()))
            state = next_state.copy()
            action = random_policy(state)
        episodes.append(episode)
    return episodes

# Generar episodios con Exploring Starts
exploring_episodes = exploring_starts(env)

In [54]:
for i in exploring_episodes[0]:
    print(i)

({'product_A': 1, 'product_B': 5}, {'product_A': 0, 'product_B': 1}, 18, {'product_A': 2, 'product_B': 6})
({'product_A': 2, 'product_B': 6}, {'product_A': 1, 'product_B': 2}, 1, {'product_A': 1, 'product_B': 8})
({'product_A': 1, 'product_B': 8}, {'product_A': 0, 'product_B': 1}, 3, {'product_A': 0, 'product_B': 9})
({'product_A': 0, 'product_B': 9}, {'product_A': 0, 'product_B': 2}, -14, {'product_A': 0, 'product_B': 10})
({'product_A': 0, 'product_B': 10}, {'product_A': 1, 'product_B': 0}, 10, {'product_A': 1, 'product_B': 9})
({'product_A': 1, 'product_B': 9}, {'product_A': 2, 'product_B': 1}, -2, {'product_A': 3, 'product_B': 9})
({'product_A': 3, 'product_B': 9}, {'product_A': 2, 'product_B': 0}, 0, {'product_A': 4, 'product_B': 9})
({'product_A': 4, 'product_B': 9}, {'product_A': 0, 'product_B': 2}, 1, {'product_A': 4, 'product_B': 9})
({'product_A': 4, 'product_B': 9}, {'product_A': 1, 'product_B': 2}, 1, {'product_A': 3, 'product_B': 10})
({'product_A': 3, 'product_B': 10}, {'

In [57]:
def epsilon_greedy_policy(state, epsilon=0.1):
    if random.random() < epsilon:
        return random_policy(state)
    else:
        return {product: 1 for product in state.keys()}

# Generar episodios usando epsilon-greedy
def generate_episode_with_epsilon(env, policy, epsilon=0.1, max_steps=10):
    state = env.reset()
    episode = []
    for _ in range(max_steps):
        action = policy(state, epsilon)
        next_state, reward = env.step(action)
        episode.append((state, action, reward, next_state.copy()))
        state = next_state.copy()
    return episode

epsilon_episodes = [generate_episode_with_epsilon(env, epsilon_greedy_policy) for _ in range(100)]


In [58]:
for i in epsilon_episodes[0]:
    print(i)

({'product_A': 3, 'product_B': 1}, {'product_A': 1, 'product_B': 1}, 38, {'product_A': 3, 'product_B': 2})
({'product_A': 3, 'product_B': 2}, {'product_A': 1, 'product_B': 1}, 23, {'product_A': 2, 'product_B': 2})
({'product_A': 2, 'product_B': 2}, {'product_A': 1, 'product_B': 1}, 8, {'product_A': 1, 'product_B': 3})
({'product_A': 1, 'product_B': 3}, {'product_A': 1, 'product_B': 1}, 38, {'product_A': 0, 'product_B': 2})
({'product_A': 0, 'product_B': 2}, {'product_A': 1, 'product_B': 1}, 13, {'product_A': 0, 'product_B': 2})
({'product_A': 0, 'product_B': 2}, {'product_A': 1, 'product_B': 1}, -2, {'product_A': 0, 'product_B': 3})
({'product_A': 0, 'product_B': 3}, {'product_A': 1, 'product_B': 1}, 13, {'product_A': 0, 'product_B': 3})
({'product_A': 0, 'product_B': 3}, {'product_A': 1, 'product_B': 1}, 3, {'product_A': 1, 'product_B': 3})
({'product_A': 1, 'product_B': 3}, {'product_A': 1, 'product_B': 1}, 18, {'product_A': 2, 'product_B': 2})
({'product_A': 2, 'product_B': 2}, {'pr

In [61]:
# Política objetivo (siempre reponer a nivel 1 por ejemplo)
def target_policy(state):
    return {product: 1 for product in state.keys()}

# Generar episodios usando aprendizaje off-policy
def generate_off_policy_episode(env, behavior_policy, target_policy, epsilon=0.1, max_steps=10):
    state = env.reset()
    episode = []
    for _ in range(max_steps):
        action = behavior_policy(state, epsilon)
        next_state, reward = env.step(action)
        episode.append((state, action, reward, next_state.copy()))
        state = next_state.copy()
    return episode

off_policy_episodes = [generate_off_policy_episode(env, epsilon_greedy_policy, target_policy) for _ in range(100)]

In [62]:
for i in off_policy_episodes[0]:
    print(i)

({'product_A': 7, 'product_B': 3}, {'product_A': 1, 'product_B': 1}, 18, {'product_A': 8, 'product_B': 0})
({'product_A': 8, 'product_B': 0}, {'product_A': 1, 'product_B': 1}, -2, {'product_A': 8, 'product_B': 1})
({'product_A': 8, 'product_B': 1}, {'product_A': 1, 'product_B': 1}, 18, {'product_A': 9, 'product_B': 0})
({'product_A': 9, 'product_B': 0}, {'product_A': 1, 'product_B': 1}, -2, {'product_A': 9, 'product_B': 1})
({'product_A': 9, 'product_B': 1}, {'product_A': 1, 'product_B': 2}, 1, {'product_A': 8, 'product_B': 3})
({'product_A': 8, 'product_B': 3}, {'product_A': 1, 'product_B': 1}, 23, {'product_A': 7, 'product_B': 3})
({'product_A': 7, 'product_B': 3}, {'product_A': 1, 'product_B': 1}, 3, {'product_A': 8, 'product_B': 3})
({'product_A': 8, 'product_B': 3}, {'product_A': 1, 'product_B': 1}, 13, {'product_A': 8, 'product_B': 3})
({'product_A': 8, 'product_B': 3}, {'product_A': 1, 'product_B': 1}, 8, {'product_A': 7, 'product_B': 4})
({'product_A': 7, 'product_B': 4}, {'pro

# Preguntas a responder

1. ¿Cuál es el valor estimado de mantener diferentes niveles de existencias para cada producto?

El valor estimado de mantener diferentes niveles de existencias para cada producto se puede saber a traves de la funcion valor accion "Q". Esta determina el valor de un estado y un accion, esto en el problema determininaría el valor para mantener diferentes niveles de existencias para cada prodcuto.

2. ¿Cómo afecta el valor epsilon en la política blanda al rendimiento?

El valor de epsilon en una politica blanda determina que tanto se explorara o bien que tanto se explotara. Para un valor epsilon alto, por ejemplo de 0.9, determina que el 90% del tiempo se tomará una acción aleatoria antes de la optima, dando como resultado un estado al que no se ha llegado antes y de esta manera cambiando la politica. Es importante tomar esto en cuenta debido a que puede determinar que tan bien funcionará la politica para lo que se este intentando de abordar y si se podrá llegar a algun optimo debido a esto.  

3. ¿Cuál es el impacto de utilizar el aprendizaje fuera de la política en comparación con el aprendizaje dentro de la política?

Mientras que al utilizar un aprendizaje dentro de la politica ofrece una opción suboptima de la acción a tomar, sin embargo puede darse el caso que su exploración puede ser limitada por lo que no llegue a aprender algo optimo. Por otro lado la fuera de politica aprende a seguir la politica que esta estudiando mientras que esta tomando acciones utilizando una distinta politica. Al lograr esto, se puede observar que tan diferentes son las 2 politicas y bien que tan buena es la politica que se esta estudiando para un estado determinado. Un ejemplo parecido a ello podrian ser los amiibos de smash bros, los cuales cuando se ponen en modo de imitación imitan la forma de jugar de la persona que esta jugando lo cual se parece a un aprendizaje fuera de politica. 